In [1]:
import tensorflow as tf
import numpy as np
from glob import glob
from scipy.misc import imread

#### Data import

In [2]:
def get_name(filepath):
    return filepath.split('/')[-1]

n_cracked   = len(glob('./Data/170729_Panel_Solos_Resized/CrackedScores/*.png'))
n_query     = 3

cracked_score_fps   = glob('./Data/170729_Panel_Solos_Resized/CrackedScores/*.png')
uncracked_score_fps = glob('./Data/170729_Panel_Solos_Resized/UncrackedScores/*.png') 

image_filepaths     = (['./Data/170729_Panel_Solos_Resized/Cracked/' + get_name(fp) 
                         for fp in cracked_score_fps] +
                       ['./Data/170729_Panel_Solos_Resized/Uncracked/' + get_name(fp)
                         for fp in uncracked_score_fps])

score_filepaths  = ( cracked_score_fps +
                      uncracked_score_fps )

scores  = np.expand_dims(np.array(
                [imread(fp) for fp in score_filepaths], dtype = 'float32'), axis = -1) / 255
scores  = np.concatenate([scores, 1 - scores], axis = -1) # One-hot encode scores
images  = np.expand_dims(np.array(
                [imread(fp) for fp in image_filepaths], dtype = 'float32'), axis = -1)

query_ix     = np.random.randint(0, n_cracked, 3)
query_mask   = np.zeros([scores.shape[0]], dtype = bool)
query_mask[query_ix] = True

query_scores = scores[query_mask, :, :, :]
query_images = images[query_mask, :, :, :]
scores       = scores[np.logical_not(query_mask), :, :, :]
images       = images[np.logical_not(query_mask), :, :, :]

labels       = (np.sum(scores, axis = (1, 2)) > 0).squeeze()[:, 0]

print('Images and score masks successfully read.')

Images and score masks successfully read.


#### Model Definition

In [72]:
# Assume that we are receiving a batch of 40x40 pixel patches

patch_size  = 40
input_size  = patch_size**2
batch_size  = 32
n_factors   = 2

graph = tf.Graph()

with graph.as_default():
    with tf.name_scope('Inputs'):
        X      = tf.placeholder(tf.float32, [batch_size, input_size])
    
    with tf.name_scope('Variables'):
        mu    = tf.Variable(tf.truncated_normal(
                                shape = [input_size]))
        Sigma = tf.Variable(tf.eye(input_size))
        Phi   = tf.Variable(tf.truncated_normal(
                                shape = [input_size, n_factors]))
        
    def E_step(X, Phi, Sigma, mu):
        """
            Returns a [batch_size, n_factors] matrix.
        
            X     : [batch_size, input_size] matrix of observations.
            Phi   : [input_size, n_factors] matrix of factors.
            Sigma : [input_size, input_size] diagonal covariance matrix.
            mu    : [input_size] vector located at x's mean.
        """
        inv_Sigma = tf.diag(tf.divide(1, tf.diag_part(Sigma))) # [input_size, input_size]
        tmp1      = tf.tensordot(tf.transpose(Phi), inv_Sigma, axes = [[1], [0]]) # [n_factors, input_size]
        tmp2      = tf.tensordot(tmp1, Phi, axes = [[1], [0]]) # [n_factors, n_factors]
        tmp3      = tf.matrix_inverse(tmp2 + tf.eye(n_factors)) #[n_factors, n_factors]
        tmp4      = tf.tensordot(tmp3, tf.tensordot(tf.transpose(Phi), inv_Sigma, axes = [[1], [0]]),
                                 axes = [[1], [0]]) # [n_factors, input_size]
        Eh        = tf.tensordot(X - mu, tf.transpose(tmp4), axes = [[1], [0]]) # [batch_size, n_factors]
        outer     = [None]*batch_size
        for i in range(batch_size):
            outer[i] = tmp3 + tf.tensordot(tf.slice(Eh, [i, 0], [1, -1]),
                                           tf.slice(Eh, [i, 0], [1, -1]),
                                           axes = [[1], [1]])
        Ehh       = tf.stack(outer, axis = 0)
        return Eh, Ehh
    
    def M_step(X, mu, Phi, Eh, Ehh):
        """
            Returns ops to update the model's parameters.
            Updates are performed analytically.
            
            mu    : [input_size]
            Sigma : [input_size, input_size]
            Phi   : [input_size, n_factors]
        """
        mu      = update_mu(X)
        Phi     = update_Phi(X, mu, Eh, Ehh)
        Sigma   = update_Sigma(X, mu, Phi, Eh)
        return mu, Phi, Sigma
    
    def update_mu(X):
        """
            Returns an op that updates mean.
            
            X          : [batch_size, input_size]
            mu_hat     : [input_size]
            batch_size : scalar
        """
        mu_hat = tf.reduce_mean(X, axis = 0)
        return mu_hat
    
    def update_Phi(X, mu, Eh, Ehh):
        """
            Returns an op that updates matrix of factors.
            
            X          : [batch_size, input_size]
            mu         : [input_size]
            batch_size : scalar
            Eh         : [batch_size, n_factors]
            Ehh        : [batch_size, n_factors, n_factors]
        """
        
        # Compute outer products over all expectation/example pairs
        outer = [None]*batch_size
        for i in range(batch_size):
            outer[i] = tf.tensordot(tf.transpose(tf.slice(X, begin = [i, 0], size = [1, -1])), # [n_factors, 1]
                                    tf.slice(Eh, begin = [i, 0], size = [1, -1]),              # [input_size, 1]
                                    axes = [[1], [0]])                                         # [input_size, n_factors]
            
        tmp3     = tf.stack(outer, axis = 0)
        tmp2     = tf.matrix_inverse(tf.reduce_sum(Ehh, axis = 0)) # [n_factors, n_factors]
        Phi_hat  = tf.tensordot(tf.reduce_sum(tmp3, axis = 0), tmp2, axes = [[1], [0]])
        return Phi_hat
    
    def update_Sigma(X, mu, Phi, Eh):
        """
            Returns an op that updates the diagonal covariance matrix.
            
            X          : [batch_size, input_size]
            mu         : [input_size]
            batch_size : scalar
            Eh         : [batch_size, n_factors]
            Phi        : [input_size, n_factors]
            Sigma_hat  : [input_size, input_size]
        """
        squares = [None]*batch_size
        
        for i in range(batch_size):
            xi          = tf.slice(X, begin = [i, 0], size = [1, -1])
            Ehi         = tf.transpose(tf.slice(Eh, begin = [i, 0], size = [1, -1]))
            deviation   = tf.transpose(xi - mu) # [input_size, 1]
            tmp1        = tf.pow(deviation, 2) # [input_size, 1]
            tmp2        = tf.tensordot(Phi, Ehi, axes = [[1], [0]]) # [input_size, 1]
            squares[i]  = tf.transpose(tmp1 - tf.multiply(tmp2, deviation)) # [input_size, 1]

        var_diag  = tf.reduce_mean(tf.stack(squares, axis = 0), axis = 0)
        Sigma_hat = tf.diag(tf.squeeze(var_diag))
        return Sigma_hat
        
    with tf.name_scope('Optimization'):
        with tf.name_scope('E_step'):
            Eh, Ehh   = E_step(X, Phi, Sigma, mu) # [batch_size, n_factors] - Each row corresponds to a single example
            
        with tf.name_scope('M_step'):
            with tf.control_dependencies([Eh, Ehh]):
                mu, Phi, Sigma = M_step(X, mu, Phi, Eh, Ehh)
        
        optimize       = tf.group(mu, Phi, Sigma)
        
    with tf.name_scope('Inference'):
        covar         = tf.tensordot(Phi, tf.transpose(Phi), axes = [[1], [0]]) + Sigma
        mvn           = tf.contrib.distributions.MultivariateNormalFullCovariance(loc = mu,
                                                                                  covariance_matrix = covar)
        probabilities = mvn.prob(X) # [batch_size] vector

In [73]:
class Minibatch:
    def __init__(self, minibatch_size, patch_size):
        self.candidate_ix   = np.array(np.nonzero(scores[:,:,:,0] > 0.5)).T
        self.patch_size     = patch_size
        self.minibatch_size = minibatch_size
    
    def get(self):
        rand_ints    = np.random.randint(0, self.candidate_ix.shape[0],self.minibatch_size)
        chosen_ix    = self.candidate_ix[rand_ints, :]
        patches      = self._get_patches(images, chosen_ix)
        flat_patches = np.reshape(patches, newshape = [self.minibatch_size, -1])
        return flat_patches
    
    def _get_patches(self, images, ix):
        """
            Accepts a stack of images [N, R, Co, Ch], indices to sample at, patch sizes.
            
            Returns a stack of image patches
        """
        ps         = self.patch_size
        patches    = np.zeros([ix.shape[0], patch_size, patch_size, images.shape[3]])

        for j, i in enumerate(ix):
            if (j == 0) or np.any(images[i[0], :, :, :] != img):
                img  = images[i[0], :, :, :]
                pimg = np.pad(img, pad_width = [[ps, ps], [ps, ps], [0, 0]],
                              mode = 'reflect')
            patches[j, :, :, :] =  pimg[i[1] + ps // 2 : i[1] + (3 * ps // 2),
                                        i[2] + ps // 2 : i[2] + (3 * ps // 2), :]

        return patches
        

In [77]:
mb = Minibatch(batch_size, patch_size)
training_steps = 10

bat = mb.get()

with tf.Session(graph = graph) as session:
    session.run(tf.global_variables_initializer())
    print('Model initialized.\n')
    
    for i in range(training_steps):
        fd = { X :  bat}
        _, phi  = session.run([optimize, Phi], feed_dict = fd)

        if i % 1 == 0:
            print('Step {:^3d}: Phi = {}'.format(i, phi))

Model initialized.

Step  0 : Phi = [[-1513351.625  1513353.375]
 [-1508007.     1508008.875]
 [-1493993.375  1493995.125]
 ..., 
 [-1627272.5    1627273.875]
 [-1675145.375  1675146.875]
 [-1674264.     1674265.5  ]]
Step  1 : Phi = [[-1513351.625  1513353.375]
 [-1508007.     1508008.875]
 [-1493993.375  1493995.125]
 ..., 
 [-1627272.5    1627273.875]
 [-1675145.375  1675146.875]
 [-1674264.     1674265.5  ]]
Step  2 : Phi = [[-1513351.625  1513353.375]
 [-1508007.     1508008.875]
 [-1493993.375  1493995.125]
 ..., 
 [-1627272.5    1627273.875]
 [-1675145.375  1675146.875]
 [-1674264.     1674265.5  ]]
Step  3 : Phi = [[-1513351.625  1513353.375]
 [-1508007.     1508008.875]
 [-1493993.375  1493995.125]
 ..., 
 [-1627272.5    1627273.875]
 [-1675145.375  1675146.875]
 [-1674264.     1674265.5  ]]
Step  4 : Phi = [[-1513351.625  1513353.375]
 [-1508007.     1508008.875]
 [-1493993.375  1493995.125]
 ..., 
 [-1627272.5    1627273.875]
 [-1675145.375  1675146.875]
 [-1674264.     1674

# WHY ISNT IT WORKING